In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os
import urllib

driver = '{ODBC Driver 17 for SQL Server}'
server = os.environ["SERVER"]
database = os.environ["DATABASE"]
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]

odbc_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;UID='+username+';DATABASE='+ database + ';PWD='+ password
connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote_plus(odbc_str)

In [3]:
# temp_conn = 'mssql+pyodbc:///?odbc_connect=DRIVER={SQL Server};SERVER=th1.database.windows.net;DATABASE=Th-1-pgsql;Trusted_Connection=yes;'

In [3]:
print("CONN STRING")
print(connect_str)

CONN STRING
mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3Dth1.database.windows.net%3BPORT%3D1433%3BUID%3Dth1-admin%3BDATABASE%3DTh-1-pgsql%3BPWD%3DThreSh0lD%4001


In [13]:
from langchain_community.utilities import SQLDatabase
import sqlalchemy

In [14]:
engine = sqlalchemy.create_engine(connect_str)
db = SQLDatabase(engine, schema="SalesLT")

In [15]:
# db = SQLDatabase.from_uri(connect_str)
print(db.dialect)
print(db.get_usable_table_names())

mssql
['Address', 'Customer', 'CustomerAddress', 'Product', 'ProductCategory', 'ProductDescription', 'ProductModel', 'ProductModelProductDescription', 'SalesOrderDetail', 'SalesOrderHeader']


In [16]:
db.run("SELECT * FROM SalesLT.Address;")

'[(9, \'8713 Yosemite Ct.\', None, \'Bothell\', \'Washington\', \'United States\', \'98011\', \'268AF621-76D7-4C78-9441-144FD139821A\', datetime.datetime(2006, 7, 1, 0, 0)), (11, \'1318 Lasalle Street\', None, \'Bothell\', \'Washington\', \'United States\', \'98011\', \'981B3303-ACA2-49C7-9A96-FB670785B269\', datetime.datetime(2007, 4, 1, 0, 0)), (25, \'9178 Jumping St.\', None, \'Dallas\', \'Texas\', \'United States\', \'75201\', \'C8DF3BD9-48F0-4654-A8DD-14A67A84D3C6\', datetime.datetime(2006, 9, 1, 0, 0)), (28, \'9228 Via Del Sol\', None, \'Phoenix\', \'Arizona\', \'United States\', \'85004\', \'12AE5EE1-FC3E-468B-9B92-3B970B169774\', datetime.datetime(2005, 9, 1, 0, 0)), (32, \'26910 Indela Road\', None, \'Montreal\', \'Quebec\', \'Canada\', \'H1Y 2H5\', \'84A95F62-3AE8-4E7E-BBD5-5A6F00CD982D\', datetime.datetime(2006, 8, 1, 0, 0)), (185, \'2681 Eagle Peak\', None, \'Bellevue\', \'Washington\', \'United States\', \'98004\', \'7BCCF442-2268-46CC-8472-14C44C14E98C\', datetime.datetim

### Tools for the Agent

In [17]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_sql_agent



In [18]:
toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(model="gpt-4"))

agent_executor = create_sql_agent(
   llm=ChatOpenAI(model="gpt-4"),
   toolkit=toolkit,
   verbose=True,
)

In [ ]:
agent_executor.run("What is the total number of customers in the database?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Address, Customer, CustomerAddress, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeaderThe Customer table looks relevant to the query "What is the total number of customers in the database?". Let's check the schema for the Customer table to know its structure.
Action: sql_db_schema
Action Input: Customer
CREATE TABLE [SalesLT].[Customer] (
	[CustomerID] INTEGER NOT NULL IDENTITY(1,1), 
	[NameStyle] BIT NOT NULL DEFAULT ((0)), 
	[Title] NVARCHAR(8) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[FirstName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[MiddleName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[LastName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Suffix] NVARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[CompanyName] NVARCHAR(128) COLLATE SQL_Latin1_General_CP

'The total number of customers in the database is 847.'

In [20]:
agent_executor.run("List top 5 popular products with their description")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Address, Customer, CustomerAddress, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeaderThe tables which are most likely to contain necessary information are 'Product' and 'SalesOrderDetail'. I will look at their schema.
Action: sql_db_schema
Action Input: Product, SalesOrderDetail
CREATE TABLE [SalesLT].[Product] (
	[ProductID] INTEGER NOT NULL IDENTITY(1,1), 
	[Name] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[ProductNumber] NVARCHAR(25) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Color] NVARCHAR(15) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[StandardCost] MONEY NOT NULL, 
	[ListPrice] MONEY NOT NULL, 
	[Size] NVARCHAR(5) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Weight] DECIMAL(8, 2) NULL, 
	[ProductCategoryID] INTEGER NULL, 
	[ProductModelID] INTEGER NULL, 
	[SellStartDate] DATETIME NOT NULL

'The top 5 popular products with their quantities ordered are:\n1. Sport-100 Helmet, Red - 370 units\n2. Classic Vest, S - 87 units\n3. Short-Sleeve Classic Jersey, XL - 57 units\n4. Bike Wash - Dissolver - 55 units\n5. Water Bottle - 30 oz. - 54 units'

In [21]:
agent_executor.run("Display all columns of products in a table")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""Address, Customer, CustomerAddress, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeaderThe relevant table to this question is 'Product'. Next, I need to find out the columns in the Product table.
Action: sql_db_schema
Action Input: Product
CREATE TABLE [SalesLT].[Product] (
	[ProductID] INTEGER NOT NULL IDENTITY(1,1), 
	[Name] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[ProductNumber] NVARCHAR(25) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Color] NVARCHAR(15) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[StandardCost] MONEY NOT NULL, 
	[ListPrice] MONEY NOT NULL, 
	[Size] NVARCHAR(5) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Weight] DECIMAL(8, 2) NULL, 
	[ProductCategoryID] INTEGER NULL, 
	[ProductModelID] INTEGER NULL, 
	[SellStartDate] DATETIME NOT NULL, 
	[SellEndDate] DATETIME NULL, 
	[Discon

'Here are the top 10 products with their details:\n\n1. Product ID 680, "HL Road Frame - Black, 58", with standard cost 1059.31 and list price 1431.50. Full details: [\'FR-R92B-58\', \'Black\', \'58\', 1016.04, 18, 6, datetime.datetime(2002, 6, 1, 0, 0), \'no_image_available_small.gif\', \'43DD68D6-14A4-461F-9069-55309D90EA7E\', datetime.datetime(2008, 3, 11, 10, 1, 36, 827000)].\n2. Product ID 706, "HL Road Frame - Red, 58", with standard cost 1059.31 and list price 1431.50. Full details: [\'FR-R92R-58\', \'Red\', \'58\', 1016.04, 18, 6, datetime.datetime(2002, 6, 1, 0, 0), \'no_image_available_small.gif\', \'9540FF17-2712-4C90-A3D1-8CE5568B2462\', datetime.datetime(2008, 3, 11, 10, 1, 36, 827000)].\n3. Product ID 707, "Sport-100 Helmet, Red", with standard cost 13.0863 and list price 34.9900. Full details: [\'HL-U509-R\', \'Red\', 35, 33, datetime.datetime(2005, 7, 1, 0, 0), \'no_image_available_small.gif\', \'2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712\', datetime.datetime(2008, 3, 11, 10,